1. Find two land registry records that are likely to be errors. 
In each case, provide a potential explanation 

In [ ]:
import pandas as pd

# Download and read Land Registry data
data_path = "pp-complete.csv"
df = pd.read_csv(data_path)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
column_names = ['transaction_id', 'price', 'date', 'postcode', 'property_type', 'old_new', 'duration', 'paon', 'saon', 'street', 'locality', 'town_city', 'district', 'county', 'ppd_category_type', 'record_status']
df.columns = column_names

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'])

Plot the data to check if there is any outliers

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Set the style of the visualization
sns.set(style="whitegrid")

# Create the boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['price'])

# Set the title and labels
plt.title('Boxplot of Prices')
plt.xlabel('Price')

# Show the plot
plt.show()



From the graphical analysis of the dataset, it is evident that there are some anomalies in the data that could be indicative of fake transactions or potential data entry errors. These anomalies must be addressed to ensure the accuracy and reliability of the dataset.

In [ ]:

# Filter for potential outliers
outliers = df[(df["price"] < df["price"].quantile(0.01))]

# Analyze outliers and provide explanations

# Print outliers and potential explanations
for index, row in outliers.iterrows():
  print(f"Record ID: {index}, Price: {row['price']}, Explanation: Possible data entry error or a fake transaction.")


To remove the spotted anomalies, I calculated the 0.01 quantile of the price column, which helps identify and filter out transactions with unusually low prices. For instance, transactions such as Record ID: 29212254 with a price of 500 and Record ID: 29212252 with a price of 3000 fall below this threshold and are flagged as anomalies. By removing these low-priced entries, we can eliminate possible erroneous or fraudulent records, thereby refining the dataset for more accurate and meaningful analysis.

In [ ]:
# all_postcodes = df['postcode']
# for i in all_postcodes:
#     if len(i) != 8:
#         print(i)
rows_with_nan = df[df['postcode'].isna()]
print(rows_with_nan)

In addition to anomalies in the price data, there is a possibility of errors in the postcodes within the dataset. These errors, however, do not necessarily invalidate the entire entry. A postcode might be entered incorrectly due to typographical errors, or it might be missing altogether, yet other details such as the price and property characteristics could still be accurate and valuable. Such errors in postcodes should be carefully addressed rather than leading to the outright rejection of these entries. But in a less time frame I will be deleting those entries if there is any.

In [ ]:
## delete rows with NaN values
print(len(df))
df = df.dropna(subset=['postcode'])
print(len(df))


UK postcodes is in the range of 5 to 8 characters long, so including space it must be under the 9 chars and greater than 5 chars.

In [ ]:
df['is_valid_postcode'] = df['postcode'].apply(lambda x: ((len(x) < 9) & (len(x) > 5)))

In [ ]:
print(len(df[df['is_valid_postcode'] == True]))
print(len(df[df['is_valid_postcode'] == False]))

In [ ]:
df[df['is_valid_postcode'] == False]

2. Complete the following table by calculating the number of sales and average sale price for all London Boroughs in 2023:

Required columns:
London Borough, 
Count of sales in 2023, 
Average sale price in 2023

In [ ]:
# show df of couty == Greater London and town_city == london and locality == BOROUGH
df[(df['county'] == 'GREATER LONDON') & (df['locality'] == 'LONDON')]['district'].unique()
# df[df['county']=='GREATER LONDON']['district'].unique()
# rows_with_london = df[df.applymap(lambda x: x == 'BOROUGH').any(axis=1)]
# rows_with_london

The above are the all London Boroughs

In [ ]:
# Filter data for London Boroughs in 2023
london_boroughs = list(df[(df['county'] == 'GREATER LONDON') & (df['locality'] == 'LONDON')]['district'].unique())
borough_data = df[(df["district"].isin(london_boroughs)) & (df["date"].dt.year == 2023)]

# Calculate sales count and average price
borough_sales = borough_data.groupby("district").agg(count=("transaction_id", "count"), 
                                                    avg_price=("price", "mean"))

# Print results as a table
print(borough_sales.to_string())

In [ ]:
borough_sales_df = borough_sales.reset_index()
col_new_names = ['London Borough', 'Count of sales in 2023', 'Average sale price in 2023']
borough_sales_df.columns = col_new_names
borough_sales_df

3. Count the number of new build Flats sold in each UK region since the start of 2020 

In [ ]:
# Filter data for new build flats since 2020
new_flats = df[(df["property_type"] == "F") & (df["date"].dt.year >= 2020) & (df["old_new"] == "N")]

# Group and count sales by region
region_sales = new_flats.groupby("county").agg(count=("transaction_id", "count"))

# Print results as a table
print(region_sales.to_string())


To count the number of new build flats sold in each UK region since the start of 2020, I followed a systematic approach. First, I filtered the dataset based on the property_type column, selecting only those records where the property type is specified as 'F' for flats. Next, I further refined the dataset by including only those transactions with a date later than 2020. This was achieved by filtering the date column to include records from 2020 onwards. To ensure that only new build flats were counted, I filtered the dataset once more using the old_new column, selecting only those entries marked with 'N' for new builds. Finaly, to categorize the data by UK regions, I grouped the filtered records by the county column, which corresponds to the different regions within the UK. This comprehensive filtering and grouping process allowed me to accurately count the number of new build flats sold in each UK region since the start of 2020, providing valuable insights into the distribution and volume of new residential property sales across the country.

4. Plot the number of sales per week since the start of 2020 as a line chart Discuss the chart, and provide potential explanations for any patterns or anomalies 


In [ ]:
since2020datadf = df[(df["date"].dt.year >= 2020)]
# Group sales by week and count occurrences
weekly_sales = since2020datadf.resample("W-Sun", on="date")["transaction_id"].count()

# Plot sales per week
weekly_sales.plot(kind="line", figsize=(10, 6))
plt.xlabel("Week")
plt.ylabel("Number of Sales")
plt.title("Sales per Week since 2020")
plt.show()


The analyzed sales per week chart reveals a general trend of decreasing sales over the years, punctuated by several notable anomalies where sales figures either peaked or dropped significantly. These fluctuations in sales can often be attributed to a range of external factors. For example, economic conditions such as inflation, changes in fiscal or housing policies, and seasonal variations can cause deviations from the general trend. The year 2024, for instance, shows the lowest sales, which could be reflective of recent economic pressures or shifts in the property market. Conversely, the middle of 2021 experienced the highest sales, which might be due to a temporary surge in market activity or favorable conditions at that time. The decreasing trend in sales might also be related to broader issues such as inflationary pressures and rising land and property prices, which can reduce affordability and dampen market activity. Understanding these anomalies within the broader context of economic and market conditions can provide valuable insights into the factors influencing property sales over time.

5. Plot a histogram of sale prices and discuss which distribution best represents the data Feel free to transform the data before plotting, but explain your reasoning if you choose to do so.

In [ ]:
import numpy as np 

# Plot price distribution as a histogram
plt.hist(df["price"], bins=20)
plt.xlabel("Sale Price")
plt.ylabel("Frequency")
plt.title("Distribution of Sale Prices")
plt.show()

# Apply log transformation to sale prices
df["log_price"] = df["price"].apply(lambda x: np.log(x))

# Plot log price distribution as a histogram
plt.hist(df["log_price"], bins=20)
plt.xlabel("Log Sale Price")
plt.ylabel("Frequency")
plt.title("Distribution of Log Sale Prices")
plt.show()


The distribution of sale prices is right-skewed, with a long tail on the right side. This indicates that there are some high-value properties that are significantly more expensive than the rest. To better visualize the distribution, I have appllied a log transformation to the sale prices and plot the histogram again. The log-transformed sale prices now follow a more normal distribution, which makes it easier to identify patterns and trends in the data. The log transformation helps to reduce the impact of extreme values and outliers, making the distribution more symmetrical and easier to interpret.

6. Using the BNG tiles and an appropriate scale, plot a map showing the number of sales per 10km grid square 

In [ ]:
# !pip install geopandas>= 0.8.0

I am unable to perform the below tasks with the complete land registry dataset because the memory error issue. I have tried it on the aws cloud as well but I need to pay more and purchase the resources in aws. So I will use the monthly land registry data for the following tasks.

In [ ]:
import pandas as pd
from geopandas import GeoDataFrame, read_file
import matplotlib.pyplot as plt

# Load Land Registry data (replace with your processed data)
df = pd.read_csv("pp-monthly-update-new-version.csv")

column_names = ['transaction_id', 'price', 'date', 'postcode', 'property_type', 'old_new', 'duration', 'paon', 'saon', 'street', 'locality', 'town_city', 'district', 'county', 'ppd_category_type', 'record_status']
df.columns = column_names

# Load BNG Grid data
bng_grid = read_file("OSGB_Grids-master\GeoJSON\OSGB_Grid_5km.geojson")
print(bng_grid.head())

In [ ]:
bng_grid.head()

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pyproj import Transformer

# Load the OSGB dataset from JSON file
osgb_tiles = bng_grid
osgb_tiles = osgb_tiles.to_crs(epsg=27700)

# Load the sales data from CSV file
sales_data = df

# Creating latitude and longitude columns by merging the sales data with the postcode coordinates
postcode_coords = pd.read_csv('NSPL_2021_MAY_2024/Data/NSPL21_MAY_2024_UK.csv')
postcode_coords_df = postcode_coords[['pcds', 'lat', 'long']]
new_col_names = ['postcode', 'latitude', 'longitude']
postcode_coords_df.columns = new_col_names
sales_data = sales_data.merge(postcode_coords_df, on='postcode')

Here, have a Dataset with 'pcds', 'lat', 'long' columns which then merged with the land registry data which have postcode column. Applied merging on postcode.

In [ ]:
sales_data.head()

In [ ]:
# #Examine Geometry Data: 
# gdf_sales.is_valid.sum() 
# osgb_tiles.is_valid.sum()

# #Verify CRS: 
# gdf_sales.crs 
# osgb_tiles.crs

In [ ]:
# Convert the sales data to a GeoDataFrame
gdf_sales = gpd.GeoDataFrame(sales_data, geometry=gpd.points_from_xy(sales_data.longitude, sales_data.latitude))

# Transformer to convert WGS84 to OSGB36
transformer = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)

# Convert sales coordinates to OSGB36
gdf_sales['geometry'] = gdf_sales['geometry'].apply(lambda geom: Point(transformer.transform(geom.x, geom.y)))
gdf_sales.set_crs("EPSG:27700", inplace=True)


In [ ]:
# Perform spatial join to map sales to OSGB tiles
gdf_sales_with_tiles = gpd.sjoin(gdf_sales, osgb_tiles, how="left", predicate="within")
gdf_sales_with_tiles.head()

In [ ]:
# Aggregate sales count per OSGB tile
sales_per_tile = gdf_sales_with_tiles.groupby('TILE_NAME').size().reset_index(name='sales_count')
sales_per_tile

In [ ]:
# Merge sales count with OSGB tiles
osgb_tiles = osgb_tiles.merge(sales_per_tile, on='TILE_NAME', how='left').fillna(0)
osgb_tiles

In [ ]:
# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
osgb_tiles.plot(column='sales_count', ax=ax, legend=True, cmap='OrRd', legend_kwds={'label': "Number of Sales"})

# Set plot title and labels
plt.title('Number of Sales per 10km Grid Square')
plt.xlabel('Easting')
plt.ylabel('Northing')

# Show the plot
plt.show()


In [ ]:
# Calculate average sale price per grid square
avg_price_per_tile = gdf_sales_with_tiles.groupby('TILE_NAME')['price'].mean().reset_index(name = 'avg_price')
avg_price_per_tile.columns

7. Plot a map showing the average sale price per 10km grid square 

In [ ]:

# Merge average price with OSGB tiles
osgb_tiles = osgb_tiles.merge(avg_price_per_tile, on='TILE_NAME', how='left').fillna(0)

# Plot the map (replace 'sales_count' with 'avg_price')
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
osgb_tiles.plot(column='avg_price', ax=ax, legend=True, cmap='plasma') 
plt.title('Average Sale Price per 10km Grid Square')
plt.xlabel('Easting')
plt.ylabel('Northing')
plt.show()

8. Comment on your findings, 
Given everything you have learned from the exercises above, discuss the following model.
In your discussion, provide an approximate R square value that you would expect from the
model
How would you improve the model?
What range of R square would you be happy with?

Price ~ β ∗ Y ear + ∑i=1 α +
nα
i ∑i=1 γ +
nγ
i ε

Where: β
is the coefficient for the year of sale αi and γi are dummy variables corresponding to postcode area and property type
respectively

The current model estimates the relationship between sale price and year, while also accounting for categorical variables such as postcode area and property type. With these categorical variables in play, an R-squared value between 0.6 and 0.8 is considered reasonable, reflecting a moderately good fit depending on the complexity of the data. To enhance the model's performance, several improvements can be made. First, it is beneficial to incorporate additional features that could significantly influence sale prices, such as property size, number of bedrooms and bathrooms, lot size, property age, and condition or renovation status. Including these features can provide a more comprehensive view of what drives property values. 

Second, exploring interaction terms between features, like the interplay between year and property type or postcode area and property type, may reveal more nuanced relationships and improve model accuracy. Advanced modeling techniques such as random forests or gradient boosting machines could be employed if the data exhibits complex, non-linear relationships. These methods are adept at capturing intricate patterns without requiring explicit specification of interactions. 

Additionally, regularization techniques like Lasso or Ridge regression could be useful in managing multicollinearity and mitigating overfitting. While aiming for a higher R-squared is desirable, an acceptable R-squared range should be considered within the context of the data and the model's purpose. In real estate, where numerous factors impact property values, an R-squared in the 0.6 to 0.8 range might be quite acceptable, particularly if the model remains interpretable and avoids overfitting. Balancing model complexity with predictive accuracy and interpretability is key to achieving a robust and useful model.

9. Energy Performance Certificates (EPCs) are published for each property transaction
recorded by the Land Registry. The data schema for these is available here

I. Which fields from this dataset would be useful in determining sale price?
II. How would you approach the task of joining the EPC database with the Land Registry?
N.B. We don't expect you to actually join the databases - but please provide as
much detail as possible about how you would approach the task.

To analyze the relationship between property sale prices and energy efficiency as recorded in Energy Performance Certificates (EPCs), it is useful to leverage specific fields from EPCs that are potentially influential in determining sale prices. The EPC data schema provides several pertinent fields. For instance, the **Energy Rating** is a key field as it indicates the property's energy efficiency; properties with higher ratings (e.g., A or B) might command higher sale prices due to anticipated savings on energy costs. Similarly, the **Property Type** field is valuable if EPC data categorizes properties in a way that aligns with the Land Registry’s classification. This alignment allows for meaningful comparisons of energy efficiency within property types. Additionally, **Floor Area** is crucial because the size of the property can affect both its energy efficiency and its market value. Larger properties with high energy ratings might fetch higher prices compared to smaller ones with the same rating, impacting the sale price on a price-per-unit-area basis. The **Main Heating Fuel** field indicates the type of fuel used for heating, with renewable or more efficient fuels potentially making the property more attractive to buyers and thus affecting its sale price. Finally, the **Age of Boiler** could be relevant as newer, more efficient boilers generally lower running costs, potentially influencing the sale price positively.

To integrate and analyze data from both Land Registry and EPC databases, start by identifying a common unique identifier that links property transactions across the datasets. This identifier could be a property reference number, address, or a combination of fields. If the datasets are housed in different systems, a Database Management System (DBMS) or tools like SQL can be used to execute a join operation based on this identifier. As my preference, if the datasets are in CSV or Excel format, I will use programming languages Python, with libraries like Pandas, to facilitate the merging process.

During the merging process, it is essential to address potential issues such as duplicate entries or missing values in the common identifier field. Techniques such as deduplication or imputation should be employed to handle these inconsistencies. Data cleaning is crucial to ensure consistency across datasets, including standardizing formats for fields like address or property type to achieve a more accurate join.

Once the datasets are successfully merged, the combined dataset can be analyzed to explore how EPC features influence sale prices. Analytical techniques such as correlation analysis or regression modeling can be used to quantify the relationships between EPC features and property sale prices, revealing how energy efficiency might impact market values. This comprehensive approach provides insights into the economic implications of energy efficiency on property sales.